In [124]:
import pandas as pd
import numpy as np

import requests

from ast import literal_eval

from tqdm import tqdm
tqdm.pandas()

import time

import urlexpander

In [2]:
art_data = pd.read_csv('../../data/webpage_data/full_flipside_data.csv')

In [3]:
art_data.head()

,article_url,date,title,left_sum,right_sum,linked_arts
0,https:/theflipside.io/archives/checks-in-the-mail,"March 20, 2020",Checks In The Mail,The left supports cash payments and argues tha...,The right is generally supportive of helping c...,['https://apnews.com/0373a78b0aacc49d1f6bd8901...
1,https:/theflipside.io/archives/eos-regarding-t...,"January 26, 2021",EOs Regarding Transgender Rights,The left is supportive of both policies.,The right is critical of both policies.,['https://apnews.com/article/biden-reverse-ban...
2,https:/theflipside.io/archives/facebook-and-br...,"April 2, 2019",Facebook and Breaking Up Big Tech,"The left is skeptical of Zuckerberg’s motives,...",The right is disturbed by the free speech impl...,['https://www.washingtonpost.com/opinions/mark...
3,https:/theflipside.io/archives/general-electio...,"October 28, 2020",General Election Update,The left is optimistic about Biden’s chances.,The right is cautiously optimistic about Trump...,['https://theflipside.us15.list-manage.com/tra...
4,https:/theflipside.io/archives/impeachment-hea...,"November 22, 2019",Impeachment Hearings Continue,"The left supports impeachment, arguing that th...","The right opposes impeachment, arguing that th...",['https://theflipside.us15.list-manage.com/tra...


In [4]:
art_data['linked_arts'] = art_data['linked_arts'].str.replace('\n', ',')

In [5]:
art_data['news_list'] = art_data['linked_arts'].apply(literal_eval)

In [6]:
art_data['num_news'] = art_data['news_list'].apply(len)

# Shortening Num URLS 

In [7]:
art_data['first_news_list'] = art_data['news_list'].progress_apply(lambda l: l[:5])

100%|████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 196058.96it/s]


In [8]:
art_data.to_csv('./temp_art_data.csv', index = None)

# Getting Real URLS 

In [9]:
def get_real_url(url):
    try:
        resp = requests.get(url, allow_redirects = True)
    except ConnectionError as e:
        continue
    
    try:
        return resp.url
    except:
        return 'no_source'
    
def get_real_url_list(news_list):
    news_list_real = []
    for news in news_list:
        if 'theflipside' in news:
            news_list_real.append(get_real_url(news))
        else:
            news_list_real.append(news)
    
    return news_list_real

In [10]:
all_news = [news for newslist in art_data['first_news_list'].values for news in newslist]
fs_news = [news for news in all_news if 'theflipside' in news]

In [11]:
url_map = {}

In [12]:
start = 0

In [31]:
for i, fs_url in enumerate(fs_news[start:]):
    rurl = get_real_url(fs_url)
    url_map[fs_url] = rurl
    time.sleep(0.15)
    start += 1
    print(f'Finished {i + 1} of {len(fs_news) - 903}', end = '\r')

In [27]:
fs_news[start]

'https://theflipside.us15.list-manage.com/track/click?u=281c839c990f42b374467ae5f&id=bf72f389fa&e=6f16cb1c23'

In [28]:
url_map[fs_news[start]] = 'www.wisconsingrassroots.net/wisconsin_election_officials_field_guide'

In [32]:
start

1191

In [30]:
start += 1

In [33]:
len(url_map)

1170

In [35]:
with open('url_map.txt', 'w') as f:
    f.write(str(url_map))

In [37]:
def get_real_urls(old_urls):
    old_urls = [url_map[url] if url in url_map.keys() else url for url in old_urls]
    return old_urls

In [38]:
art_data['first_news_real'] = art_data['first_news_list'].progress_apply(get_real_urls)

100%|████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 162548.11it/s]


In [40]:
art_data.to_csv('../../data/webpage_data/full_flipside_data_proc.csv', index = None)

In [159]:
all_first_news = [news for newslist in art_data['first_news_real'].values for news in newslist]

In [160]:
len(all_first_news)

4907

In [161]:
all_first_news[0]

'https://apnews.com/0373a78b0aacc49d1f6bd8901c8ccc67'

In [162]:
import urllib
#     url_query = urllib.parse.quote_plus(search_query)

In [163]:
TWITTER_BASE = 'https://twitter.com/search?q=%s&src=typed_query'

def get_twitter_search(news_url):
    return TWITTER_BASE % (urllib.parse.quote_plus(news_url))

In [164]:
first_news_tw = '\n'.join([get_twitter_search(fnews) for fnews in all_first_news])

In [53]:
with open('../../data/twitter_data/query_urls.txt', 'w') as f:
    f.write(first_news_tw)

In [55]:
len(first_news_tw.split('\n'))

4907

In [56]:
batch1 = pd.read_csv('../../data/twitter_data/tweets_batch1.csv')

C:\Users\ndeas\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (20,21,22,23,24,25,26,31,32,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,200,201,203,204,206,207,209,210) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
url_cols = [f'urls/{i}/expanded_url' for i in range(10)]

In [71]:
list(batch1.columns)

['conversation_id',
 'created_at',
 'favorite_count',
 'full_text',
 'hashtags/0',
 'hashtags/1',
 'hashtags/2',
 'hashtags/3',
 'hashtags/4',
 'hashtags/5',
 'hashtags/6',
 'hashtags/7',
 'hashtags/8',
 'hashtags/9',
 'hashtags/10',
 'hashtags/11',
 'hashtags/12',
 'hashtags/13',
 'hashtags/14',
 'hashtags/15',
 'hashtags/16',
 'hashtags/17',
 'hashtags/18',
 'hashtags/19',
 'hashtags/20',
 'hashtags/21',
 'hashtags/22',
 'id',
 'reply_count',
 'retweet_count',
 'symbols/0',
 'symbols/1',
 'symbols/2',
 'url',
 'urls/0/display_url',
 'urls/0/expanded_url',
 'urls/0/url',
 'urls/1/display_url',
 'urls/1/expanded_url',
 'urls/1/url',
 'urls/2/display_url',
 'urls/2/expanded_url',
 'urls/2/url',
 'urls/3/display_url',
 'urls/3/expanded_url',
 'urls/3/url',
 'urls/4/display_url',
 'urls/4/expanded_url',
 'urls/4/url',
 'urls/5/display_url',
 'urls/5/expanded_url',
 'urls/5/url',
 'urls/6/display_url',
 'urls/6/expanded_url',
 'urls/6/url',
 'urls/7/display_url',
 'urls/7/expanded_url',
 '

In [72]:
batch1[url_cols]

,urls/0/expanded_url,urls/1/expanded_url,urls/2/expanded_url,urls/3/expanded_url,urls/4/expanded_url,urls/5/expanded_url,urls/6/expanded_url,urls/7/expanded_url,urls/8/expanded_url,urls/9/expanded_url
0,https://apnews.com/0373a78b0aacc49d1f6bd8901c8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://apnews.com/0373a78b0aacc49d1f6bd8901c8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://apnews.com/0373a78b0aacc49d1f6bd8901c8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://apnews.com/0373a78b0aacc49d1f6bd8901c8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://apnews.com/0373a78b0aacc49d1f6bd8901c8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
14497,https://www.cnbc.com/2021/10/27/how-the-democr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14498,https://cnb.cx/3pY75VF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,http://dlvr.it/SBQ8qJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14500,https://www.cnbc.com/2021/10/27/how-the-democr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
tweet_urls = np.hstack(batch1[url_cols].values)

In [139]:
tweet_urls = tweet_urls[tweet_urls == tweet_urls]

In [140]:
resolved = urlexpander.expand(tweet_urls,
                            chunksize = 1280,
                            n_workers = 20,
                            cache_file = 'tmp.json')

In [145]:
resolved = urlexpander.expand(tweet_urls,
#                              chunksize = 1280,
                            n_workers = 20,
                            cache_file = 'tmp.json')

In [148]:
res_map = {key: val for key, val in zip(tweet_urls, resolved)}

In [149]:
def get_real_url(row):
    urls = row[url_cols]
    urls = urls.dropna()
    urls = [res_map[url] if url in res_map.keys() else url for url in urls.values]
    urls = [url[:url.find('?')] if '?' in url else url for url in urls]
    urls = [url for url in urls if url in all_first_news]
    if len(urls) > 0:
        return urls[0]
    return None

In [150]:
batch1_res = batch1.progress_apply(get_real_url, axis = 1)

100%|██████████████████████████████████████████████████████████████████████████| 14502/14502 [00:07<00:00, 2060.97it/s]


In [152]:
batch1['resolved_url'] = batch1_res

In [166]:
all_resolved = [news for news in batch1['resolved_url'].values]

In [168]:
miss_queries = set(all_first_news).difference(set(all_resolved))

In [170]:
TWITTER_BASE = 'https://twitter.com/search?q=%s&src=typed_query'

def get_twitter_search(news_url):
    return TWITTER_BASE % (urllib.parse.quote_plus(news_url))

In [171]:
first_news_tw2 = '\n'.join([get_twitter_search(fnews) for fnews in miss_queries])

In [175]:
with open('../../data/twitter_data/query_urls2.txt', 'w') as f:
    f.write(first_news_tw2)

In [176]:
len(first_news_tw2.split('\n'))

4592